In [11]:
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import dash
from dash import Dash, dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import warnings

In [12]:
# Unternehmen des S&P100 laden, Berkshire Hathaway entfernen und zufällige 25 Unternehmen auswählen

url = "https://en.wikipedia.org/wiki/S%26P_100"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
section = soup.find('span', {'id': 'Components'}).parent
table = section.find_next_sibling('table')

tickers = []

for row in table.find_all('tr')[1:]:
    cols = row.find_all('td')
    if len(cols) > 1:
        ticker = cols[0].text.strip()
        company_name = cols[1].text.strip() 
        tickers.append({
            'Symbol': ticker,
            'Name': company_name
        })

df = pd.DataFrame(tickers)
df = df[~df['Symbol'].isin(['BRK.A', 'BRK.B'])]

ps = df.sample(n=25, random_state=105943, replace=False)

In [13]:
# täglichen Daten des adjusted Close Preises und Volume von Beginn des Jahres 2021 bis heute laden

warnings.simplefilter(action='ignore', category=FutureWarning)

start_date = "2021-01-01"
end_date = datetime.now().strftime('%Y-%m-%d')

trading_data = {}

for ticker_symbol in ps['Symbol']:
    ticker = yf.Ticker(ticker_symbol)
    hist_data = ticker.history(start=start_date, end=end_date)
    data = hist_data[['Close', 'Volume']]
    data = data.rename(columns={'Close': 'Adjusted Close'})
    trading_data[ticker_symbol] = data

In [14]:
# Kurs des S&P 500 laden

warnings.simplefilter(action='ignore', category=FutureWarning)

sp500_data = yf.Ticker("^GSPC")
sp500_hist_data = sp500_data.history(start=start_date, end=end_date)

sp500_data = sp500_hist_data[['Close', 'Volume']].rename(columns={'Close': 'SP500 Adjusted Close', 'Volume': 'SP500 Volume'})

In [15]:
# täglichen diskreten und log-Renditen des adjusted Close Preises berechnen

for ticker_symbol, data in trading_data.items():
    data['Discrete Return'] = data['Adjusted Close'].pct_change()
    data['Log Return'] = np.log(data['Adjusted Close'] / data['Adjusted Close'].shift(1))

sp500_data['SP500 Discrete Return'] = sp500_data['SP500 Adjusted Close'].pct_change()
sp500_data['SP500 Log Return'] = np.log(sp500_data['SP500 Adjusted Close'] / sp500_data['SP500 Adjusted Close'].shift(1))

Es empfiehlt sich den S&P500 dem S&P100 vorzuziehen, da ersterer deutlich mehr Unternehmen in Betracht zieht und daher auf eine größere Datenbasis zurückgreift. Dadurch ist das Spektrum der Sektoren der betrachteten Unternehmen breiter und demnach deutlich diversifizierter. Der S&P500 liefert daher einen umfassenderen Überblick über die wirtschaftliche Lage und gewichtet Ausreißer (sowohl positiv, als auch negativ) nicht zu stark.

In [16]:
# Informationen über Unternehmen laden

for index, row in ps.iterrows():
    ticker = yf.Ticker(row['Symbol'])
    
    info = ticker.info
    
    ps.at[index, 'Mitarbeiteranzahl'] = info.get('fullTimeEmployees')
    ps.at[index, 'Sektor'] = info.get('sector')
    ps.at[index, 'Marktkapitalisierung'] = info.get('marketCap')
    ps.at[index, 'Website'] = info.get('website')
    if info.get('website'):
        domain = info.get('website').split('//')[-1].split('/')[0]
        logo_url = f"https://logo.clearbit.com/{domain}"
        ps.at[index, 'Logo URL'] = logo_url
    else:
        ps.at[index, 'Logo URL'] = None

In [17]:
# pro Kalenderjahr den Intercept und Koeffizienten gemäß des CAPM schätzen

capm_estimates = pd.DataFrame(columns=['Ticker', 'Year', 'Beta', 'Intercept'])

years = [2021, 2022, (2023, 2024)]

for ticker_symbol, data in trading_data.items():
    for year in years:

        if isinstance(year, tuple):
            data_filtered = data[data.index.year.isin(year)]
            year_label = f"{year[0]}-{year[1]}"
        else:
            data_filtered = data[data.index.year == year]
            year_label = str(year)
        
        market_returns = sp500_data.loc[data_filtered.index, 'SP500 Log Return']
        stock_returns = data_filtered['Log Return']
        
        if not market_returns.empty and not stock_returns.empty:
            joint_df = pd.DataFrame({'Stock Return': stock_returns, 'Market Return': market_returns}).dropna()

            if not joint_df.empty:
                covariance = np.cov(joint_df['Stock Return'], joint_df['Market Return'])[0][1]
                market_variance = np.var(joint_df['Market Return'], ddof=1)
                beta = covariance / market_variance
                
                alpha = np.mean(joint_df['Stock Return']) - beta * np.mean(joint_df['Market Return'])
                
                capm_estimates = capm_estimates._append({'Ticker': ticker_symbol, 'Year': year_label, 'Beta': beta, 'Intercept': alpha}, ignore_index=True)

In [18]:
annual_returns = pd.DataFrame(columns=['Ticker', 'Annual Return', 'Jahr'])

years = [2021, 2022, (2023, 2024)]

for ticker, data in trading_data.items():
    for year in years:
        if isinstance(year, tuple):
            yearly_data = data[data.index.year.isin(year)]
            year_label = f"{year[0]}-{year[1]}"
        else:
            yearly_data = data[data.index.year == year]
            year_label = str(year)

        if not yearly_data.empty:
            annual_return = np.exp(yearly_data['Log Return'].sum()) - 1
            annual_returns = annual_returns._append({'Ticker': ticker, 'Annual Return': annual_return, 'Jahr': year_label}, ignore_index=True)

for year in years:
    if isinstance(year, tuple):
        yearly_data = sp500_data[sp500_data.index.year.isin(year)]
        year_label = f"{year[0]}-{year[1]}"
    else:
        yearly_data = sp500_data[sp500_data.index.year == year]
        year_label = str(year)

    if not yearly_data.empty:
        annual_return = np.exp(yearly_data['SP500 Log Return'].sum()) - 1
        annual_returns = annual_returns._append({'Ticker': '^GSPC', 'Annual Return': annual_return, 'Jahr': year_label}, ignore_index=True)

In [19]:
aggregated_data = pd.DataFrame(columns=['Jahr', 'Bestes Unternehmen', 'Beste Rendite', 'Beta (Best)', 'Schlechtestes Unternehmen', 'Schlechteste Rendite', 'Beta (Schlecht)', 'Mittleres systematisches Risiko', 'Mittleres idiosynkratisches Risiko', 'S&P 500 Rendite'])

marktvarianz = sp500_data['SP500 Log Return'].var()

if 'Year' not in capm_estimates.columns:
    capm_estimates['Year'] = capm_estimates.index.year

marktvarianz = sp500_data['SP500 Log Return'].var()

unique_years = sorted(annual_returns['Jahr'].unique())

if 2023 in unique_years and 2024 in unique_years:
    
    unique_years.remove(2024)

for year in unique_years:
    
    if year == 2023:
        year_data = annual_returns[(annual_returns['Jahr'] == 2023) | (annual_returns['Jahr'] == 2024)]
        year_label = "2023-2024"
    else:
        year_data = annual_returns[annual_returns['Jahr'] == year]
        year_label = str(year)
        
    year_data = annual_returns[annual_returns['Jahr'] == year]
    
    best_return = year_data['Annual Return'].max()
    worst_return = year_data['Annual Return'].min()
    
    best_company = year_data.loc[year_data['Annual Return'].idxmax(), 'Ticker']
    worst_company = year_data.loc[year_data['Annual Return'].idxmin(), 'Ticker']
    
    best_beta = capm_estimates.loc[capm_estimates['Ticker'] == best_company, 'Beta'].values[0]
    worst_beta = capm_estimates.loc[capm_estimates['Ticker'] == worst_company, 'Beta'].values[0]
    
    year_capm_estimates = capm_estimates[capm_estimates['Year'] == str(year)]
    
    mean_beta = year_capm_estimates['Beta'].mean()
    
    best_idiosyncratic_risk = None
    worst_idiosyncratic_risk = None
    
    for ticker in year_capm_estimates['Ticker']:
        company_data = trading_data[ticker]
        company_log_returns = company_data['Log Return']

        company_var = company_log_returns.var()

        beta = year_capm_estimates.loc[year_capm_estimates['Ticker'] == ticker, 'Beta'].values[0]

        idiosyncratic_risk = company_var - (beta ** 2 * marktvarianz)
        year_capm_estimates.loc[year_capm_estimates['Ticker'] == ticker, 'Idiosyncratic Risk'] = idiosyncratic_risk
        
        systematic_risk = beta ** 2 * marktvarianz
        year_capm_estimates.loc[year_capm_estimates['Ticker'] == ticker, 'Systematic Risk'] = systematic_risk
        
        if ticker == best_company:
            best_idiosyncratic_risk = idiosyncratic_risk
            best_systematic_risk = systematic_risk
        elif ticker == worst_company:
            worst_idiosyncratic_risk = idiosyncratic_risk
            worst_systematic_risk = systematic_risk

    mean_idiosyncratic_risk = year_capm_estimates['Idiosyncratic Risk'].mean()
    mean_systematic_risk = year_capm_estimates['Systematic Risk'].mean()
    
    sp500_return = year_data.loc[year_data['Ticker'] == '^GSPC', 'Annual Return'].values[0]
    
    new_row = {
        'Jahr': year_label,
        'Bestes Unternehmen': best_company,
        'Beste Rendite': f"{best_return*100:.2f}%",
        'Beta (Best)': best_beta,
        'Systematisches Risiko (Best)': best_systematic_risk,
        'Idiosynkratisches Risiko (Best)': best_idiosyncratic_risk,
        'Schlechtestes Unternehmen': worst_company,
        'Schlechteste Rendite': f"{worst_return*100:.2f}%",
        'Beta (Schlecht)': worst_beta,
        'Systematisches Risiko (Schlecht)': worst_systematic_risk,
        'Idiosynkratisches Risiko (Schlecht)': worst_idiosyncratic_risk,
        'Mittleres systematisches Risiko': mean_systematic_risk,
        'Mittleres idiosynkratisches Risiko': mean_idiosyncratic_risk,
        'S&P 500 Rendite': f"{sp500_return*100:.2f}%"
    }
    
    aggregated_data = aggregated_data._append(new_row, ignore_index=True)

data_list = []

# Dataframe zu Liste umwandeln

for index, row in aggregated_data.iterrows():
    row_dict = {
        'Jahr': row['Jahr'],
        'Bestes Unternehmen': row['Bestes Unternehmen'],
        'Beste Rendite': row['Beste Rendite'],
        'Beta (Best)': row['Beta (Best)'],
        'Systematisches Risiko (Best)': row['Systematisches Risiko (Best)'],
        'Idiosynkratisches Risiko (Best)': row['Idiosynkratisches Risiko (Best)'],
        'Schlechtestes Unternehmen': row['Schlechtestes Unternehmen'],
        'Schlechteste Rendite': row['Schlechteste Rendite'],
        'Beta (Schlecht)': row['Beta (Schlecht)'],
        'Systematisches Risiko (Schlecht)': row['Systematisches Risiko (Schlecht)'],
        'Idiosynkratisches Risiko (Schlecht)': row['Idiosynkratisches Risiko (Schlecht)'],
        'Mittleres systematisches Risiko': row['Mittleres systematisches Risiko'],
        'Mittleres idiosynkratisches Risiko': row['Mittleres idiosynkratisches Risiko'],
        'S&P 500 Rendite': row['S&P 500 Rendite']
    }

    data_list.append(row_dict)

/var/folders/60/xz452__n5316w2wlt9y7ryrr0000gn/T/ipykernel_77492/2126988597.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/60/xz452__n5316w2wlt9y7ryrr0000gn/T/ipykernel_77492/2126988597.py:55: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/60/xz452__n5316w2wlt9y7ryrr0000gn/T/ipykernel_77492/2126988597.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [20]:
app = dash.Dash(__name__)

# Dash-App Layout

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Gesamtübersicht', style={'font-family': 'Arial'}, children=[
            html.Div([
                dcc.Graph(id='sp500-graph'),
                dcc.RangeSlider(
                    id='year-slider',
                    min=sp500_data.index.year.min(),
                    max=sp500_data.index.year.max(),
                    value=[sp500_data.index.year.min(), sp500_data.index.year.max()],
                    marks={str(year): str(year) for year in sp500_data.index.year.unique()},
                    step=None
                ),
                
                html.Br(),
                
                dash_table.DataTable(
                    id='capm-table',
                    columns=[
                        {'name': 'Jahr', 'id': 'Jahr'},
                        {'name': 'Bestes Unternehmen', 'id': 'Bestes Unternehmen'},
                        {'name': 'Beste Rendite', 'id': 'Beste Rendite'},
                        {'name': 'Beta (Best)', 'id': 'Beta (Best)'},
                        {'name': 'Systematisches Risiko (Best)', 'id': 'Systematisches Risiko (Best)'},
                        {'name': 'Idiosynkratisches Risiko (Best)', 'id': 'Idiosynkratisches Risiko (Best)'},
                        {'name': 'Schlechtestes Unternehmen', 'id': 'Schlechtestes Unternehmen'},
                        {'name': 'Schlechteste Rendite', 'id': 'Schlechteste Rendite'},
                        {'name': 'Beta (Schlecht)', 'id': 'Beta (Schlecht)'},
                        {'name': 'Systematisches Risiko (Schlecht)', 'id': 'Systematisches Risiko (Schlecht)'},
                        {'name': 'Idiosynkratisches Risiko (Schlecht)', 'id': 'Idiosynkratisches Risiko (Schlecht)'},
                        {'name': 'Mittleres systematisches Risiko', 'id': 'Mittleres systematisches Risiko'},
                        {'name': 'Mittleres idiosynkratisches Risiko', 'id': 'Mittleres idiosynkratisches Risiko'},
                        {'name': 'S&P 500 Rendite', 'id': 'S&P 500 Rendite'},
                    ],
                    data=data_list,
                    style_table={'overflowX': 'auto', 'font-family': 'Arial'},
                    page_size=10
                )
            ])
        ]),
        dcc.Tab(label='Spezifische Unternehmen', style={'font-family': 'Arial'}, children=[
            html.Div([
                html.Br(),
                dcc.Dropdown(
                    id='company-dropdown',
                    options=[{'label': row['Name'], 'value': row['Symbol']} for index, row in ps.iterrows()],
                    value=ps['Symbol'].iloc[0]
                ),
                html.Br(),

                html.Div(id='company-details'),
                
                dcc.Graph(id='company-stock-graph'),
                
                html.Div([
                    html.Button('Zeige Quantile', id='quantile-btn', n_clicks=0, 
                                style={'margin': '10px', 'padding': '10px', 'fontSize': '16px'}),
                    html.Button('Zeige Standardabweichung', id='std-btn', n_clicks=0, 
                                style={'margin': '10px', 'padding': '10px', 'fontSize': '16px'}),
                ], style={'textAlign': 'center'}),
                dcc.Graph(id='daily-returns-graph'),
            ], style={'font-family': 'Arial'})
        ]),
    ])
], style = {'font-family': 'Arial'})

# Adjustierter Schlusskurs des S&P500 mit Range-Slider

@app.callback(
    Output('sp500-graph', 'figure'),
    [Input('year-slider', 'value')]
)
def update_graph(selected_years):
    filtered_data = sp500_data[(sp500_data.index.year >= selected_years[0]) & (sp500_data.index.year <= selected_years[1])]
    figure = {
        'data': [
            go.Scatter(
                x=filtered_data.index,
                y=filtered_data['SP500 Adjusted Close'],
                mode='lines'
            )
        ],
        'layout': {
            'title': 'S&P 500 Verlauf',
            'xaxis': {'title': 'Datum'},
            'yaxis': {'title': 'Adjustierter Schlusskurs'},
        }
    }
    return figure

# Unternehmens Informationen

@app.callback(
    Output('company-details', 'children'),
    [Input('company-dropdown', 'value')]
)
def update_company_details(ticker):
    company_row = ps[ps['Symbol'] == ticker].iloc[0]
    return html.Div([
        html.Div([
            html.Div([
                html.H3(f"{company_row['Name']} ({company_row['Symbol']})"),
                html.P(f"Sektor: {company_row['Sektor']}"),
                html.P(f"Anzahl der Mitarbeiter: {company_row['Mitarbeiteranzahl']}"),
                html.P(f"Marktkapitalisierung: $ {company_row['Marktkapitalisierung']}"),
                html.A(f"Website: {company_row['Website']}", href=company_row['Website'], target="_blank"),
            ], style={'width': '70%', 'display': 'inline-block', 'verticalAlign': 'top', 'font-family': 'Arial'}),

            html.Div([
                html.Img(src=company_row['Logo URL'], style={'height':'180px'})
            ], style={'width': '30%', 'display': 'inline-block', 'textAlign': 'right'}),
        ], style={'display': 'flex', 'flexDirection': 'row'}),
    ])

# adjusted Close Preis des Unternehmens im Vergleich zu S&P500 

@app.callback(
    Output('company-stock-graph', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_graph(selected_ticker):

    company_data = trading_data[selected_ticker]

    sp500_data_filtered = sp500_data.loc[company_data.index]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=company_data.index, y=company_data['Adjusted Close'],
                             name=f'Adjustierter Schlusskurs ({selected_ticker})', marker=dict(color="blue")))

    fig.add_trace(go.Scatter(x=sp500_data_filtered.index, y=sp500_data_filtered['SP500 Adjusted Close'],
                             name='Adjustierter Schlusskurs (S&P 500)', marker=dict(color="red"), yaxis="y2"))

    fig.update_layout(
        title="Adjustierter Schlusskurs Vergleich",
        xaxis=dict(title="Datum"),
        yaxis=dict(title=f"Adjustierter Schlusskurs ({selected_ticker})"),
        yaxis2=dict(title="Adjustierter Schlusskurs (S&P 500)", overlaying="y", side="right"),
    )

    return fig

# Tägliche Renditen Lineplot

@app.callback(
    Output('daily-returns-graph', 'figure'),
    [Input('company-dropdown', 'value'),
     Input('quantile-btn', 'n_clicks'),
     Input('std-btn', 'n_clicks')]
)
def update_daily_returns_graph(selected_ticker, quantile_btn, std_btn):

    company_data = trading_data[selected_ticker]

    daily_returns = company_data['Adjusted Close'].pct_change()
    

    quantile_10 = daily_returns.quantile(0.1)
    quantile_90 = daily_returns.quantile(0.9)
    mean = daily_returns.mean()
    std = daily_returns.std()
    

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=daily_returns.index, y=daily_returns, mode='lines', name='Tägliche Renditen'))
    
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    # 10% und 90 %-Quantil der Renditen 
    
    if 'quantile-btn' in changed_id and quantile_btn > 0:
        fig.add_hline(y=quantile_10, line_dash="dot", line_color="green", annotation_text="10% Quantil")
        fig.add_hline(y=quantile_90, line_dash="dot", line_color="red", annotation_text="90% Quantil")
        
    # +/- eine Standardabweichung vom Mittelwert der Renditen
    
    if 'std-btn' in changed_id and std_btn > 0:
        fig.add_hline(y=mean + std, line_dash="dot", line_color="blue", annotation_text="+1 Std Abw")
        fig.add_hline(y=mean - std, line_dash="dot", line_color="orange", annotation_text="-1 Std Abw")
    
    fig.update_layout(title="Tägliche Renditen", xaxis_title="Datum", yaxis_title="Rendite")
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)